# Instructions
### 1. Upload your video file.
### 2. Run the **Libs** command.
### 3. On **Pre-config** match the `file_name` variable to the uploaded video's filename.
This var will be used for the final downloadable ZIP and GIF (if desired) files (e.g., `faces_my_example_video.zip`, `faces_my_example_video.gif`).

### 4. On **FPS** set a custom value (optional).
Do not change it to use original video's FPS.

### 5. Run the **App** script.
### 6. Run the **GIF** script (optional).

# Libs

In [18]:
!pip install opencv-python dlib tqdm

# Pre-config

In [19]:
file_name = "my_example_video.mp4"
square_size = 200   # Final saved image size in pixels (after resize)
face_padding = 0.25 # Padding (%) to include around the face crop
resize_crop = True  # Resize face fit to square size

# Frames per second (FPS)

In [20]:
import cv2

cap = cv2.VideoCapture(file_name)
original_fps = cap.get(cv2.CAP_PROP_FPS)
target_fps = original_fps

print(f"Original FPS: {original_fps}")
print(f"Target FPS: {target_fps}")

# Double check your custom FPS (if apply) meets next condition:
# 0 < target_fps <= original_fps

# Note: The less target_fps is the more frames skipped

Original FPS: 23.976023976023978
Target FPS: 23.976023976023978


# App

In [21]:
import os
import time
import dlib
import shutil
from tqdm.auto import tqdm
from google.colab import files

# ==== Setup ====
detector = dlib.get_frontal_face_detector()

output_dir = "tracked_faces"
output_zip = f"faces_{file_name.split('.', 1)[0]}"
frame_interval = max(1, int(original_fps // target_fps))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
pbar = tqdm(total=total_frames, desc="Tracking frames")

if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir)

# ==== Helpers ====
def elapsed_time(seconds):
    if not isinstance(seconds, (int, float)) or seconds < 0:
        return "Bad time value"

    if seconds < 60:
        return f"{seconds:.2f} s"
    elif seconds < 3600:
        minutes = seconds / 60
        return f"{minutes:.2f} m"
    else:
        hours = seconds / 3600
        return f"{hours:.2f} h"

def as_cover(image):
    # Get current crop dimensions
    h, w = image.shape[:2]

    # Calculate difference to crop to square (centered)
    if w > h:
        offset = (w - h) // 2
        return image[:, offset:offset + h]
    elif h > w:
        offset = (h - w) // 2
        return image[offset:offset + w, :]
    return image

# ==== App ====
def tracker(target_fps, original_fps):
  if not (0 < target_fps <= original_fps):
    target_fps = original_fps  # Fallback to full speed

  frame_step = original_fps / target_fps
  next_frame = 0.0
  frame_count = 0

  while cap.isOpened():
      ret, frame = cap.read()
      if not ret:
          break

      if frame_count >= int(next_frame):
          gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
          faces = detector(gray)

          if len(faces) > 0:
              # Use first face
              face = faces[0]
              x, y, w, h = face.left(), face.top(), face.width(), face.height()

              # Center point of face
              cx, cy = x + w // 2, y + h // 2

              # Square crop: half of face box + padding
              half = int(max(w, h) * (1 + face_padding)) // 2

              # Compute crop bounds
              left   = max(cx - half, 0)
              top    = max(cy - half, 0)
              right  = min(cx + half, frame.shape[1])
              bottom = min(cy + half, frame.shape[0])

              face_crop = frame[top:bottom, left:right]

              if resize_crop:
                  # 1:1 ratio
                  face_crop = as_cover(face_crop)
                  # Resize the potentially cropped face to the square size
                  face_crop = cv2.resize(face_crop, (square_size, square_size))

              filename = os.path.join(output_dir, f"frame_{frame_count:04d}.jpg")
              cv2.imwrite(filename, face_crop)

          next_frame += frame_step

      frame_count += 1
      pbar.update(1)  # Update progress bar by 1 frame

  cap.release()

# ==== Main ====
if __name__ == "__main__":
    start_time = time.time()
    tracker(target_fps, original_fps)
    end_time = time.time()
    print(f"Tracking done [{elapsed_time(end_time - start_time)}].")

    # ZIP
    shutil.make_archive(output_zip, 'zip', output_dir)
    print(f"{output_zip}.zip saved.")

    # Download
    files.download(f"{output_zip}.zip")

Tracking frames:   0%|          | 0/103 [00:00<?, ?it/s]

Tracking done [14.14 s].
faces_my_example_video.zip saved.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# GIF

In [22]:
from PIL import Image

# ==== Config ====
duration = int(1000 / target_fps)
input_dir = "tracked_faces"
output_gif = f"faces_{file_name.split('.', 1)[0]}.gif"

# ==== Setup ====
if not os.path.exists(output_dir):
  raise ValueError(f"{input_dir} folder not found.")

image_files = sorted([
    f for f in os.listdir(input_dir)
    if f.endswith(('.jpg', '.jpeg', '.png'))
])

if not image_files:
    raise ValueError(f"No images found in {input_dir}.")

# ==== App ====
def gifter():
    # Load images into Pillow
    frames = []
    for f in tqdm(image_files, desc="Loading frames"):
        image = Image.open(os.path.join(input_dir, f))
        frames.append(image)

    # GIF
    frames[0].save(
        output_gif,
        format='gif',
        save_all=True,
        append_images=frames[1:],
        duration=duration,
        loop=0
    )

# ==== Main ====
if __name__ == "__main__":
    start_time = time.time()
    gifter()
    end_time = time.time()
    print(f"Done [{elapsed_time(end_time - start_time)}].")
    print(f"{output_gif} saved.")

    # Download
    files.download(output_gif)

Loading frames:   0%|          | 0/85 [00:00<?, ?it/s]

Done [4.22 s].
faces_my_example_video.gif saved.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>